# PRACTICA 1 - GRIPPER FLOTANTE 
## Iniciando la simulacion de Gazebo

En primer lugar, es necesario iniciar la simulación de Gazebo para el entorno del gripper flotante. 
> **Gripper asignado:** `robotiq`  
> **Objeto asignado:** `driller_small`


Para facilitar el proceso, se ha creado un archivo de lanzamiento (launch file) que configura automáticamente el entorno de simulación con el gripper y el objeto asignados.

1. Abre una Nueva Terminal: Primero, necesitas abrir una nueva ventana de terminal en tu sistema (Ctrl + Alt + T).

2. Ejecuta el Comando de Lanzamiento: En la terminal recién abierta, ejecuta el siguiente comando:



``` shellscript
user@pc:~$ roslaunch manipulacion_pkg main_simulation.launch
```

## Configuración Jupyter Notebook

In [ ]:
from practica1.configuration import NOMBRE_GRIPPER_GAZEBO,NOMBRE_GRIPPER,NOMBRE_OBJETO,NOMBRE_OBSTACULO_GAZEBO
import manipulacion_lib 
import rospy 
import PyKDL 

# Inicializar el nodo de ROS si aún no se ha inicializado
if not rospy.get_node_uri():
    rospy.init_node('gripper_flotante_gazebo', anonymous=True, 
                    log_level=rospy.WARN)

# Creación de una instancia para controlar un gripper flotante en Gazebo
simulacion_gripper_flotante = manipulacion_lib.SimulacionGripperFlotante(
                                nombre_gripper_gazebo=NOMBRE_GRIPPER_GAZEBO)

# Esta instancia permite interactuar con un gripper en la simulación de Gazebo,
# especificando su nombre
# Independientemente del gripper asignado para la práctica, se utiliza el 
# nombre "gripper" para referenciarlo en la simulación de Gazebo.


# Obtener la pose (posición y orientación) de un objeto específico en el Gazebo
# Se obtiene la pose del objeto con respecto al sistema de referencia global (world)
pose_objeto_world = simulacion_gripper_flotante.obtener_pose_objeto(
                    nombre_objeto_gazebo=NOMBRE_OBJETO)

print("Pose objeto con respecto a world")
print(pose_objeto_world)

# Fijamos una pose relativa del gripper con respecto al objeto
# Pose del gripper con respecto al objeto, especificando una
# traslación de 0.2m en X y 0.1m en Z
pose_gripper_objeto = PyKDL.Frame(PyKDL.Rotation.Quaternion(1.0, 0, 0.0, 0.0), 
                                  PyKDL.Vector(0.2, 0, 0.1))

print("Pose gripper con respecto a objeto")
print(pose_gripper_objeto)

# Calcular la pose del gripper en el sistema de referencia global a partir 
# de su pose relativa al objeto
# Se realiza una transformación de coordenadas para obtener la pose del 
# gripper en el sistema global
pose_gripper_world = pose_objeto_world * pose_gripper_objeto
print("Pose gripper con respecto a world: ")
print( pose_gripper_world)
# Fijar la pose del gripper en la simulación de Gazebo
simulacion_gripper_flotante.fijar_pose_gripper(
                    pose_gripper_world=pose_gripper_world)
# Actualiza la posición y orientación del gripper en Gazebo según
# la pose calculada

## Búsqueda de mejor agarre
Una vez que la simulación de Gazebo esté en funcionamiento, el siguiente paso es buscar el mejor agarre para el objeto asignado utilizando el gripper flotante. Para esto, se ha desarrollado el siguiente método:

1. **Lectura del yaml de posiciones:** El primer paso es leer el archivo yaml que contiene las posiciones predefinidas para el gripper. Este archivo contiene una lista de posiciones (pose gripper + articulaciones) que el gripper puede adoptar para intentar agarrar el objeto, junto con unos valores de calidad asociados a cada posición.

2. **Ejecución de la función de agarre:** Para cada posición leída del yaml, se mueve el gripper a esa posición.

3. **Chequeo de colisiones:** Después de mover el gripper a la posición, se verifica si hay colisiones entre el gripper y los objetos del mundo. Si no hay colisiones, se considera que el agarre es posible.

4. **Evaluación de la calidad del agarre:** Si el agarre es posible (sin colisiones), se evalúa la calidad del agarre utilizando los valores definidos en el yaml. Estos valores pueden estar basados en criterios como la estabilidad del agarre, la cercanía al centro de masa del objeto, etc.

De este modo, se selecciona el agarre con la mejor calidad entre las posiciones evaluadas, lo que permitirá al robot manipular el objeto de manera eficiente y segura.


In [ ]:
from practica1.auxiliar import get_grasps_list
from pathlib import Path
from manipulacion_lib import Obstaculo, DetectorColisionesGripperFlotante

GRASP_POSES_YAML_PATH = Path.cwd() / "grasp_poses" / "grasp_poses_robotiq_driller_small.yaml"

# Definición de obstáculos
obstaculo_rojo = Obstaculo('cubo', [0.75,0,0,1,0,0,0], [0.3,1,0.6], 'obstaculo')
suelo = Obstaculo('cubo', [0,0,0,1,0,0,0], [0.01,0.003,-0.01], 'suelo')
obj_x, obj_y, obj_z = pose_objeto_world.p
obj_qx, obj_qy, obj_qz, obj_qw = pose_objeto_world.M.GetQuaternion() 
objeto = Obstaculo('cubo', [obj_x, obj_y, obj_z, obj_qx, obj_qy, obj_qz, obj_qw], [0.141,0.095,0.226], 'objeto')
obstaculos = [obstaculo_rojo, suelo, objeto]

best_grasp = None
first_best_grasp = True

# 1. Iterar sobre el yaml de posiciones
for grasp in get_grasps_list(GRASP_POSES_YAML_PATH):
    # 2. Mover gripper 
    x, y, z = grasp.pose[0:3]
    qx, qy, qz, qw = grasp.pose[3:7]

    pose_gripper_objeto = PyKDL.Frame(
        PyKDL.Rotation.Quaternion(qx, qy, qz, qw),
        PyKDL.Vector(x, y, z)
    )
    pose_gripper_world = pose_objeto_world * pose_gripper_objeto
    simulacion_gripper_flotante.fijar_pose_gripper(
                    pose_gripper_world=pose_gripper_world)

    # 3. Detectar colisiones
    detector_colisiones = DetectorColisionesGripperFlotante(NOMBRE_GRIPPER, obstaculos)

    rotation = pose_gripper_world.M
    qx, qy, qz, qw = rotation.GetQuaternion()
    pose_gripper = [pose_gripper_world.p.x(), pose_gripper_world.p.y(),
                    pose_gripper_world.p.z(),
                    qx, qy, qz, qw]
    colision = detector_colisiones.hay_colision(pose_gripper)
    print(f"Colisiona en la pose actual: {colision}", )

    if not colision:
        if first_best_grasp:
            best_grasp = grasp
            first_best_grasp = False
        else:
            if grasp.epsilon_quality > best_grasp.epsilon_quality and grasp.volume_quality > best_grasp.epsilon_quality:
                best_grasp = grasp
    
    rospy.sleep(0.5)

if not best_grasp:
    print("No se ha encontrado ningun agarre que cumpla las condiciones")
else:
    print("Mejor agarre:\n" \
          f"\t - pose: {best_grasp.pose}\n" \
          f"\t - dofs: {best_grasp.dofs}\n" \
          f"\t - epsilon_quality: {best_grasp.epsilon_quality}\n" \
          f"\t - volume_quality: {best_grasp.volume_quality}")

## Mover Gripper a posición de agarre

In [ ]:
from practica1.auxiliar import set_gripper_pos
simulacion_gripper_flotante.configurar_gripper()

if best_grasp:
    set_gripper_pos(simulacion_gripper_flotante, mode="open")
    
    x, y, z = best_grasp.pose[0:3]
    qx, qy, qz, qw = best_grasp.pose[3:7]

    pose_gripper_objeto = PyKDL.Frame(
        PyKDL.Rotation.Quaternion(qx, qy, qz, qw),
        PyKDL.Vector(x, y, z)
    )
    pose_gripper_world = pose_objeto_world * pose_gripper_objeto
    simulacion_gripper_flotante.fijar_pose_gripper(
                    pose_gripper_world=pose_gripper_world)

# Mover Articulaciones del Gripper

Usando el mejor agarre encontrado, se mueve el gripper utilizando dicha configuración. 

In [ ]:
from practica1.auxiliar import set_gripper_pos
set_gripper_pos(simulacion_gripper_flotante, mode="custom", articulaciones=best_grasp.dofs)

## Iniciar proceso de movimiento del gripper y objeto

In [ ]:
from practica1.auxiliar import generate_trajectory

SQUARE_TRAJECTORY = [(0,0,0.5),
                    (1,0,0),
                    (0,0,-0.5)]

TRIANGLE_TRAJECTORY = [(0.5,0,1),
                       (0.5,0,-1)]

initial_pose = pose_gripper_world
ini_qx, ini_qy, ini_qz, ini_qw = pose_gripper_world.M.GetQuaternion() 

for increment in TRIANGLE_TRAJECTORY:
    fin_x, fin_y, fin_z = tuple(x + y for x, y in zip(initial_pose.p, increment))
    next_pose = PyKDL.Frame(
        PyKDL.Rotation.Quaternion(ini_qx, ini_qy, ini_qz, ini_qw),  # Misma orientación
        PyKDL.Vector(fin_x, fin_y, fin_z)
    )
    print(f"Moviendo gripper a: {next_pose.p}")

    for pose in generate_trajectory(initial_pose, next_pose, num_frames=100):
        new_x, new_y, new_z = pose.p
        new_pose = PyKDL.Frame(
            PyKDL.Rotation.Quaternion(ini_qx, ini_qy, ini_qz, ini_qw),  # Misma orientación
            PyKDL.Vector(new_x, new_y, new_z)
        )
        simulacion_gripper_flotante.fijar_pose_gripper(
            pose_gripper_world=new_pose)
        rospy.sleep(0.05)

    # Update initial_pose
    initial_pose = next_pose


## Abrir gripper para soltar el objeto

In [ ]:
from practica1.auxiliar import set_gripper_pos
set_gripper_pos(simulacion_gripper_flotante, mode="open")